In [14]:
import pandas as pd
import pyshorteners
import time
import datetime

In [15]:
anuncios = pd.read_excel('data/Anuncios-2024_09_30-19_10.xlsx', sheet_name='Anúncios', header=0, skiprows=[1, 2, 3, 4, 5])
cadastrados = pd.read_csv("data/Tabela de Produtos - Página25.csv").fillna('-')
do_banco = pd.read_csv("data/2024-09-30T18-13_export.csv")
anuncios['STATUS'] = anuncios['STATUS'].str.strip()  # Remove espaços em branco no início e no fim
anuncios = anuncios.drop_duplicates(subset='ITEM_ID', keep='first')
anuncios =anuncios.drop(columns=['VARIATIONS' ,'VARIATION_ID'])
print(cadastrados.shape)
print(do_banco.shape)
print(anuncios.shape)

(218, 5)
(35, 28)
(413, 15)


In [16]:
categorias = {
    'Videogames': '001',
    'Figuras de ação': '002',
    'Consoles de videogames': '003',
    'Gamepads e joysticks': '004',
    'Figuras interativas para videogames': '005',
    'Consoles de videogames, videogames e máquinas de fliperama': '006',
    'Relógios de pulso': '007',
    'Skins para consoles e controles de videogames': '008',
    'Camisetas': '009',
    'Livros': '010',
    'Veículos em miniatura': '011',
    'Camisas de futebol': '012',
    'Câmeras e barras sensoras para consoles de videogames': '013',
    'Controles remotos multimídia para consoles de videogames': '014',
    'Tênis': '015',
    'Brinquedos e hobbies': '016',
    'Produtos não classificados': '017',
    'Controles remotos para reprodutores de vídeo': '018',
    'Óculos de realidade virtual': '019',
    'Relógios de parede': '020',
    'Relógios de bolso': '021',
    'Luminárias de mesa e escritório': '022'
}


In [17]:
anuncios.columns

Index(['ITEM_ID', 'TITLE', 'QUANTITY', 'CHANNEL', 'MARKETPLACE_PRICE',
       'MSHOPS_PRICE', 'MSHOPS_PRICE_SYNC', 'CURRENCY_ID',
       'SHIPPING_METHOD_MARKETPLACE', 'SHIPPING_METHOD_MSHOPS', 'LISTING_TYPE',
       'FEE_PER_SALE_MARKETPLACE', 'FEE_PER_SALE_MSHOPS', 'STATUS',
       'CATEGORY'],
      dtype='object')

In [18]:
with_sku = cadastrados['ad_code']
with_sku_list = with_sku.tolist()


# DataFrame com os valores desejados
with_sku_products = anuncios[anuncios['ITEM_ID'].isin(with_sku_list)].rename(columns={'ITEM_ID':'ad_code','TITLE':'name',})
with_sku_products_df = with_sku_products[['name', 'ad_code', 'STATUS','MARKETPLACE_PRICE',
       'MSHOPS_PRICE', "CATEGORY"]]
print(with_sku_products.shape)

merged_diff = with_sku_products_df.merge(cadastrados, on=['name', 'ad_code'], how='outer').drop_duplicates(subset='ad_code', keep='first')

# DataFrame com os valores desejados
without_sku_products = anuncios[~anuncios['ITEM_ID'].isin(with_sku_list)].rename(columns={'ITEM_ID':'ad_code','TITLE':'name'})
without_sku_products_df = without_sku_products[['name', 'ad_code', 'MARKETPLACE_PRICE',
       'MSHOPS_PRICE', 'STATUS', "CATEGORY"]]
print(without_sku_products_df.shape)




df_concatenated = pd.concat([merged_diff, without_sku_products_df], ignore_index=True, sort=False)
df_concatenated.shape
produtos = df_concatenated.copy()


produtos['category_id'] = produtos['CATEGORY'].map(categorias)

(212, 15)
(201, 6)


In [19]:
produtos['category_id'] = produtos['CATEGORY'].map(categorias)

In [20]:
def update_product_skus(df):
    current_year_month = datetime.datetime.now().strftime("%y%m")
    # Processa cada categoria separadamente
    for category_id in df['category_id'].unique():
        # Filtra o DataFrame pela categoria
        category_mask = df['category_id'] == category_id
        
        # Calcula o year_month, que é fixo para este exemplo
        year_month = current_year_month
        
        # Extrai os contadores existentes dos SKUs
        existing_skus = df.loc[category_mask & df['sku'].notna(), 'sku']
        counters = existing_skus.str.extract(f"^{category_id}-{year_month}-(\d{{4}})$")[0].astype(str).dropna()
        valid_counters = counters[counters.str.isnumeric()].astype(int)
        
        # Determina o próximo número de SKU
        if not valid_counters.empty:
            next_counter = valid_counters.max() + 1
        else:
            next_counter = 1
        
        # Atualiza SKUs nulos
        null_skus_indices = df.index[category_mask & df['sku'].isna()]
        for idx in null_skus_indices:
            df.at[idx, 'sku'] = f"{category_id}-{year_month}-{next_counter:04d}"
            next_counter += 1

# Atualiza os SKUs
update_product_skus(produtos)

produtos
# Preenchendo valores nulos de 'coluna1' com valores de 'coluna2'
produtos['universal_code'] = produtos['universal_code'].fillna(produtos['sku'])
to_export = produtos[['name', 'ad_code', 'sku', 'universal_code', 'STATUS', 'MARKETPLACE_PRICE', 'MSHOPS_PRICE',
       'CATEGORY', 'condition']]

In [21]:
to_export.to_csv("rela.csv",index=False)

In [ ]:
do_banco.columns

In [ ]:
to = do_banco[['titulo','ITEM_ID', 'completo',
       'manual_instrucoes', 'serial_number', 'serial_caixa']]

In [67]:
to.to_excel("conferir.xlsx", index=False)